#### Import packages

In [1]:
from utils.io import create_spark_session
from pyspark.sql import functions as F
from pyspark.sql import Window
from utils.io import (
    create_spark_session,
    provide_config,
    read_with_meta
)

#### Provide config

In [2]:
config_path = 's3://ilgazy-udacity/scripts/config.yaml'
config = provide_config(config_path)

#### Read fact and dimension tables

In [3]:
spark = create_spark_session()
gdp_per_capita = read_with_meta(spark, df_meta=config['fact']['gdp_per_capita'])
human_capital_index = read_with_meta(spark, df_meta=config['fact']['human_capital_index'])
immigration = read_with_meta(spark, df_meta=config['fact']['immigration'])
press_freedom_index = read_with_meta(spark, df_meta=config['fact']['press_freedom_index'])
us_cities_demographics = read_with_meta(spark, df_meta=config['fact']['us_cities_demographics'])
global_temperatures = read_with_meta(spark, df_meta=config['fact']['global_temperatures'])
temperatures_by_country = read_with_meta(spark, df_meta=config['fact']['temperatures_by_country'])
country = read_with_meta(spark, df_meta=config['dimension']['country'])

#### Get county names
Dataframes include with country_id are joined with country dimension table to get country names

In [4]:
gdp_per_capita = gdp_per_capita.join(country, 'country_id').drop('country_id')
human_capital_index = human_capital_index.join(country, 'country_id').drop('country_id')
press_freedom_index = press_freedom_index.join(country, 'country_id').drop('country_id')
temperatures_by_country = temperatures_by_country.join(country, 'country_id').drop('country_id')
immigration = immigration \
    .withColumnRenamed('origin_country_id', 'country_id') \
    .join(country, 'country_id').drop('country_id') \
    .withColumnRenamed('country_name', 'origin_country')

## Immigration general summary

#### Top 10 origin_country that visits US in 2016

In [5]:
top_10_origin = immigration \
    .groupBy('origin_country') \
    .count() \
    .orderBy(F.col('count').desc()) \
    .limit(10)
top_10_origin.show()

+--------------+------+
|origin_country| count|
+--------------+------+
|UNITED KINGDOM|368421|
|         JAPAN|249167|
|        FRANCE|185339|
|        MEXICO|179603|
|       GERMANY|156613|
|   SOUTH KOREA|136312|
|        BRAZIL|134907|
|     AUSTRALIA|112407|
|         INDIA|107193|
|     ARGENTINA| 75128|
+--------------+------+



#### Top 10 origin_country in terms of how long days spend in a visit

In [6]:
top_10_days_stayed = immigration \
    .withColumn('days_stayed', F.datediff('departure_date', 'arrival_date')) \
    .groupBy('origin_country').agg(F.avg('days_stayed').alias('average_days_stayed'),
                                F.count('days_stayed').alias('count')) \
    .filter(F.col('count') > 10000) \
    .orderBy(F.col('average_days_stayed').desc()) \
    .limit(10)
top_10_days_stayed.show()

+--------------+-------------------+-----+
|origin_country|average_days_stayed|count|
+--------------+-------------------+-----+
|         INDIA|  39.34789796620325|90127|
|   EL SALVADOR| 28.183021518893735|13523|
|       NIGERIA| 25.926920924282403|12507|
|      HONDURAS| 25.796694589087615|13251|
|       JAMAICA| 25.626632702834534|18451|
|     GUATEMALA| 24.914150826189587|16098|
|   PHILIPPINES| 23.671542337071347|24612|
|     VENEZUELA|  20.66965045788727|38765|
|        POLAND| 19.493722207690297|15292|
|      DOMINICA| 18.472983098826077|24022|
+--------------+-------------------+-----+



#### Types of immigrations to US in 2016

In [7]:
immigration.groupBy('visa_type').count().orderBy(F.col('count').desc()).show()

+---------+-------+
|visa_type|  count|
+---------+-------+
| PLEASURE|2384159|
| BUSINESS| 488666|
|  STUDENT|  32474|
+---------+-------+



#### Top 10 city in terms of residence address of immigrations

In [8]:
top_10_residence_city = immigration \
    .groupBy('residence_city') \
    .count() \
    .orderBy(F.col('count').desc()) \
    .limit(10)
top_10_residence_city.show()

+--------------+------+
|residence_city| count|
+--------------+------+
|       FLORIDA|616399|
|      NEW YORK|530692|
|    CALIFORNIA|414074|
|        HAWAII|157990|
|       UNKNOWN|156295|
|         TEXAS|126431|
|        NEVADA|110333|
|          GUAM| 91945|
|      ILLINOIS| 74745|
|    NEW JERSEY| 68767|
+--------------+------+



## Visa type analysis

- **Prepare rank dataset**: human_capital_index and press_freedom index datasets don't include year 2016, thus 2017 has taken.

In [9]:
human_capital_rank = human_capital_index \
    .filter(F.col('year') == 2017) \
    .withColumnRenamed('country_name', 'origin_country') \
    .drop('year', 'human_capital_index')
press_freedom_rank = press_freedom_index \
    .filter(F.col('year') == 2017) \
    .withColumnRenamed('country_name', 'origin_country') \
    .drop('year', 'press_freedom_index')
gdp_per_capita_rank = gdp_per_capita \
    .filter(F.col('year') == 2016) \
    .withColumnRenamed('country_name', 'origin_country') \
    .drop('year', 'gdp_per_capita')

#### Prepare country-visa type partitioned dataset that supported by additional stats

In [10]:
window_origin_country = Window.partitionBy('origin_country')
immigration_origin_analysis = immigration \
    .groupBy('origin_country', 'visa_type').count() \
    .withColumn('total_count', F.sum('count').over(window_origin_country)) \
    .withColumn('visa_type_weight', F.col('count') / F.col('total_count')) \
    .filter(F.col('total_count') > 10000) \
    .join(human_capital_rank, 'origin_country') \
    .join(press_freedom_rank, 'origin_country') \
    .join(gdp_per_capita_rank, 'origin_country')

#### Top 10 origin country in terms of visiting US for education purpose

Press freedom stats gives us a good opinion about the individual freedom in a country. The results below demonstrates the inverse correlation between press freedom and student immigrations. Students that live in a country without press freedom, intend to go US more for educational purposes. In addition, top 10 countries below also have below average human capital index and gdp per capita.

In [11]:
immigration_origin_analysis \
    .filter(F.col('visa_type') == 'STUDENT') \
    .orderBy(F.col('visa_type_weight').desc()).show(10)

+--------------+---------+-----+-----------+--------------------+------------------+------------------+-------------------+
|origin_country|visa_type|count|total_count|    visa_type_weight|human_capital_rank|press_freedom_rank|gdp_per_capita_rank|
+--------------+---------+-----+-----------+--------------------+------------------+------------------+-------------------+
|  SAUDI ARABIA|  STUDENT| 2010|      10883| 0.18469172103280346|                73|               162|                 60|
|        TURKEY|  STUDENT|  672|      16764| 0.04008589835361489|                53|               149|                 87|
|      THAILAND|  STUDENT|  565|      14552| 0.03882627817482133|                65|               136|                126|
|     SINGAPORE|  STUDENT|  375|      11444| 0.03276826284515903|                 1|               145|                 15|
|         INDIA|  STUDENT| 3364|     107193| 0.03138264625488605|               114|               130|                195|
|       

#### Top 10 origin country in terms of visiting US for business purpose
In contrast to student immigrations, the countries that values their people more have been the top origin countries for business-related immigrations.

In [12]:
immigration_origin_analysis \
    .filter(F.col('visa_type') == 'BUSINESS') \
    .orderBy(F.col('visa_type_weight').desc()).show(10)

+--------------+---------+-----+-----------+-------------------+------------------+------------------+-------------------+
|origin_country|visa_type|count|total_count|   visa_type_weight|human_capital_rank|press_freedom_rank|gdp_per_capita_rank|
+--------------+---------+-----+-----------+-------------------+------------------+------------------+-------------------+
|     SINGAPORE| BUSINESS| 4869|      11444| 0.4254631247815449|                 1|               145|                 15|
|       BELGIUM| BUSINESS| 6064|      20796| 0.2915945374110406|                26|                 9|                 28|
|       GERMANY| BUSINESS|44343|     156613|  0.283137415157107|                11|                16|                 27|
|        POLAND| BUSINESS| 4251|      16153| 0.2631709280009905|                30|                53|                 81|
|       IRELAND| BUSINESS| 7864|      29894| 0.2630628219709641|                 6|                14|                 10|
|       FINLAND|

## Analysis of US cities demographics

#### Top 10 cities with highest foreign-born people density

In [13]:
foreign_born_ratio = us_cities_demographics \
    .filter(F.col('foreign_born_population').isNotNull()) \
    .filter(F.col('total_population') > 500000) \
    .withColumn('foreign_born_density', F.col('foreign_born_population') / F.col('total_population')) \
    .select('state', 'city', 'foreign_born_population', 'total_population', 'foreign_born_density') \
    .dropDuplicates()
foreign_born_ratio.orderBy(F.col('foreign_born_density').desc()).show(10)

+-------------+-------------+-----------------------+----------------+--------------------+
|        state|         city|foreign_born_population|total_population|foreign_born_density|
+-------------+-------------+-----------------------+----------------+--------------------+
|   CALIFORNIA|     SAN JOSE|                 401493|         1026919|  0.3909685184517961|
|     NEW YORK|     NEW YORK|                3212500|         8550405|  0.3757131972111263|
|   CALIFORNIA|  LOS ANGELES|                1485425|         3971896|   0.373983860604608|
|   CALIFORNIA|SAN FRANCISCO|                 297199|          864816| 0.34365576030045697|
|        TEXAS|      HOUSTON|                 696210|         2298628|  0.3028806749069445|
|MASSACHUSETTS|       BOSTON|                 190123|          669469|  0.2839907449037969|
|   CALIFORNIA|    SAN DIEGO|                 373842|         1394907| 0.26800496377177835|
|        TEXAS|       DALLAS|                 326825|         1300082|   0.25138

#### Top 10 cities with highest veteran people density

In [14]:
veteran_ratio = us_cities_demographics \
    .filter(F.col('veteran_population').isNotNull()) \
    .filter(F.col('total_population') > 500000) \
    .withColumn('veteran_density', F.col('veteran_population') / F.col('total_population')) \
    .select('state', 'city', 'veteran_population', 'total_population', 'veteran_density') \
    .dropDuplicates()
veteran_ratio.orderBy(F.col('veteran_density').desc()).show(10)

+----------+--------------------+------------------+----------------+-------------------+
|     state|                city|veteran_population|total_population|    veteran_density|
+----------+--------------------+------------------+----------------+-------------------+
|   FLORIDA|        JACKSONVILLE|             75432|          868031|0.08690012223065766|
|     TEXAS|         SAN ANTONIO|            109089|         1469824|0.07421909017678308|
|   ARIZONA|              TUCSON|             38182|          531674|0.07181468343383351|
|     TEXAS|             EL PASO|             47693|          681136|0.07001979046768927|
|    NEVADA|           LAS VEGAS|             42397|          623769| 0.0679690718839827|
|NEW MEXICO|         ALBUQUERQUE|             37443|          559131|0.06696641753006004|
|CALIFORNIA|           SAN DIEGO|             92489|         1394907|0.06630477874152184|
|  OKLAHOMA|       OKLAHOMA CITY|             41843|          631263|0.06628457552557333|
|  KENTUCK

#### Top 10 states with youngest population

In [15]:
youngest_cities = us_cities_demographics \
    .filter(F.col('median_age').isNotNull()) \
    .select('state', 'city', 'median_age', 'total_population') \
    .dropDuplicates() \
    .groupBy('state') \
    .agg((F.sum(F.col('median_age') * F.col('total_population')) / F.sum('total_population')).alias('average_age'))
youngest_cities.orderBy(F.col('average_age').asc()).show(10)

+--------------------+------------------+
|               state|       average_age|
+--------------------+------------------+
|                UTAH|  30.3310593751517|
|           WISCONSIN|31.920280650605566|
|              ALASKA|              32.0|
|         MISSISSIPPI| 32.18462356242506|
|               TEXAS| 32.50986361312457|
|                IOWA|32.889554667346225|
|            ARKANSAS| 32.91582510989542|
|        NORTH DAKOTA| 32.96543353211251|
|DISTRICT OF COLUMBIA|              33.0|
|             GEORGIA| 33.11723217333165|
+--------------------+------------------+
only showing top 10 rows



## Climate change analysis

### Temperatures are constantly increasing since the beginning of 20th century

In [16]:
global_temperatures \
    .groupBy('decade').agg(F.mean('land_avg_temperature').alias('land_avg_temperature')) \
    .orderBy(F.col('decade').desc()).show(100)

+------+--------------------+
|decade|land_avg_temperature|
+------+--------------------+
| 2010s|   9.622430516613854|
| 2000s|    9.49389165242513|
| 1990s|   9.155183297395705|
| 1980s|   8.911491670211156|
| 1970s|     8.6584000090758|
| 1960s|    8.63312501013279|
| 1950s|    8.62083334326744|
| 1940s|   8.725416634480158|
| 1930s|   8.641675025224686|
| 1920s|    8.49266666173935|
| 1910s|   8.280849968393644|
| 1900s|   8.257475004593532|
| 1890s|   8.150416667262713|
| 1880s|   8.046774985392888|
| 1870s|   8.276966658234596|
| 1860s|   8.104766645282506|
| 1850s|   8.064883325000604|
| 1840s|   7.978099996348222|
| 1830s|     7.7376999715964|
| 1820s|   8.182233330607414|
| 1810s|   7.251066638032595|
| 1800s|    8.29814168761174|
| 1790s|   8.336866683264573|
| 1780s|   8.141391668468714|
| 1770s|   8.400108352055152|
| 1760s|   7.981625017523766|
| 1750s|   8.149851836747041|
+------+--------------------+



### Coldest countries by decade

In [17]:
temperatures_by_country \
    .groupBy('decade', 'country_name').agg(F.mean('avg_temperature').alias('avg_temperature')) \
    .orderBy(F.col('avg_temperature').asc()).show(10)

+------+------------+-------------------+
|decade|country_name|    avg_temperature|
+------+------------+-------------------+
| 1880s|   GREENLAND| -19.66855000456174|
| 1890s|   GREENLAND| -19.61917499601841|
| 1860s|   GREENLAND|-19.358591613173484|
| 1830s|   GREENLAND|-19.329749990006288|
| 1910s|   GREENLAND|-19.216849998633066|
| 1850s|   GREENLAND|-19.029166664679845|
| 1840s|   GREENLAND|-18.985891751448314|
| 1900s|   GREENLAND| -18.97052505550285|
| 1970s|   GREENLAND| -18.74794997324546|
| 1870s|   GREENLAND| -18.64782500664393|
+------+------------+-------------------+
only showing top 10 rows



### In parallel to global temperature trend, Greenland has been getting hotter year by year.

In [18]:
temperatures_by_country \
    .filter(F.col('country_name') == 'GREENLAND') \
    .groupBy('decade', 'country_name').agg(F.mean('avg_temperature').alias('avg_temperature')) \
    .orderBy(F.col('decade').desc()).show(100)

+------+------------+-------------------+
|decade|country_name|    avg_temperature|
+------+------------+-------------------+
| 2010s|   GREENLAND|-16.299841007387098|
| 2000s|   GREENLAND|-17.038825014450897|
| 1990s|   GREENLAND| -18.32337496081988|
| 1980s|   GREENLAND| -18.62722500960032|
| 1970s|   GREENLAND| -18.74794997324546|
| 1960s|   GREENLAND|-18.235008384287358|
| 1950s|   GREENLAND|-18.220725027720132|
| 1940s|   GREENLAND|-17.848025012016297|
| 1930s|   GREENLAND| -17.25954168746248|
| 1920s|   GREENLAND|-18.245991618931292|
| 1910s|   GREENLAND|-19.216849998633066|
| 1900s|   GREENLAND| -18.97052505550285|
| 1890s|   GREENLAND| -19.61917499601841|
| 1880s|   GREENLAND| -19.66855000456174|
| 1870s|   GREENLAND| -18.64782500664393|
| 1860s|   GREENLAND|-19.358591613173484|
| 1850s|   GREENLAND|-19.029166664679845|
| 1840s|   GREENLAND|-18.985891751448314|
| 1830s|   GREENLAND|-19.329749990006288|
| 1820s|   GREENLAND| -18.64466666394756|
+------+------------+-------------